# regression based on frequency matrix

In [1]:
## import modules
import json
import re
import pandas as pd
import nltk
import math
from math import sqrt
import sklearn
import statsmodels.api as sm
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
from textblob import TextBlob

wnl = WordNetLemmatizer() 
table = str.maketrans('', '', string.punctuation)

In [2]:
## define stopwords:since we only do regression based on frequency, we keep "not" as stop words
sr = stopwords.words('english')

In [3]:
## read review data
data_train=pd.read_csv("review_train.csv")
data_train

,business_id,date,stars,text
0,31292,2013-05-07 04:34:36,1.0,Total bill for this horrible service? Over $8G...
1,104374,2015-05-26 07:01:54,4.0,This review is for just the gym facilities as ...
2,170000,2015-12-28 05:52:45,2.0,There are better and worse Y's for sure. This...
3,51689,2013-12-28 22:45:53,1.0,"Terrible Gym, the ceiling in the mens shower r..."
4,69171,2013-04-10 17:15:32,5.0,Fitness together\n\nI can say nothing but posi...
5,178332,2014-09-25 08:55:35,3.0,This Gold's is a pretty good gym..for the pric...
6,51630,2010-08-21 06:53:33,5.0,Wow...no reviews on here yet? I am surprised.....
7,53385,2015-06-08 18:21:10,5.0,I love this place! I started my Bootcamp worko...
8,151835,2014-10-11 05:55:38,4.0,So far so good...19.99 gets you and a guest i...
9,61726,2018-01-21 14:54:04,3.0,GoodLife member for 3 years now... I personall...


In [4]:
## review processiong
review_train_clean = [[1]]*len(data_train)
##a list  with dict element,you can get a summary of frequency of one review by [i] and frequency for a specific word by [i]['word']
frequency_train_clean = [[1]]*len(data_train)  
for i in tqdm(range(len(data_train))):
    #Split into words
    x = word_tokenize(data_train.iloc[i]['text'])
    #Change n't into not
    x = ['not' if w=='n\'t' else w for w in x ]
    #Remove punctuation
    x = [w.translate(table) for w in x]
    #Remove not alphabetic
    x = [word for word in x if word.isalpha()]
    #Convert to lower case
    x = [w.lower() for w in x]
    #Remove stop words
    x = [w for w in x if not w in sr]
    ## lemmatization
    x = [wnl.lemmatize(w) for w in x]
    review_train_clean[i] = x
    frequency_train_clean[i] = collections.Counter(x)

100%|██████████| 35033/35033 [00:59<00:00, 593.44it/s]


In [ ]:
review_train_clean = [" ".join(item) for item in review_train_clean]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(review_train_clean)
x = vectorizer.transform(review_train_clean)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from math import sqrt
response = pd.DataFrame(data_train)['stars']
X_train, X_test, Y_train, Y_test = train_test_split(x, response, test_size=0.2, random_state=0)
mod = LogisticRegression(solver = 'newton-cg',multi_class='multinomial')
mod.fit(X_train, Y_train)
y_pred = mod.predict(X_test)
count_misclassified = (Y_test != y_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(Y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))
print("RMSE = ", sqrt(sum(np.square(Y_test-y_pred)) / len(y_pred)))

# regression based on tfidf value

In [6]:
##compute how many reviews in which this key appears
def f_appear_num(word,dat):
    num=0
    for i in range(0,len(dat)):
        d = dat[i]
        if word in d == True:
            num = num+1
    return num
def tfidf(revw,dat):
    ## input is a list of words for one review
    text = {}
    for key in revw:
        text[key] = text.get(key, 0) + 1
    tf = dict()
    idf = dict()
    tfidfvalue = dict()
    for key in text:
        num = f_appear_num(key,dat)
        tf[key] = text[key]/len(text)
        idf[key] = math.log(len(dat)/(1+num)) 
        tfidfvalue[key]= tf[key]*idf[key]
    return tfidfvalue

In [ ]:
tfidflist = []
for i in tqdm(range(len(review_train_clean))):
    tfidflist.append(tfidfvalue(review_train_clean[i],review_train_clean))

In [ ]:
predictors_tfidf = pd.DataFrame(tfidflist)
predictors.fillna(0)
## logistic regression
model_logit_tfidf = sm.Logit(response, predictors_tfidf)
model_logit_tfidf.fit()
## coefficients and significance
clf.summary()